In [91]:
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
from mindquantum.core.circuit import Circuit
import mindspore as ms
from mindquantum.simulator import  Simulator
from mindquantum.core.gates import GroupedPauli
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRConvertible,PRGenerator,ParameterResolver
from DQAS_tool import generate_pauli_string,one_hot
from mindquantum.core.gates import RotPauliString
from mindquantum.core.gates import UnivMathGate
from mindspore import Tensor, ops
from mindquantum.core.circuit import UN
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
from mindquantum.framework import MQLayer
from mindspore.nn import  TrainOneStepCell
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore import Parameter, Tensor
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
num_layer = 3
# 定义标准差和形状
stddev = 0.02
shape_nnp = (num_layer, 8)
shape_stp = (num_layer, 12)

rtype = np.float64
ctype = np.complex128
# 使用 numpy 生成随机数矩阵
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
#Operator Pool
unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(8)]
bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(8,12)]

In [2]:
from mindquantum.core.gates import  gene_univ_parameterized_gate

In [5]:
for index_op,each_op in enumerate(bound_opeartor_pool):
    print(index_op,each_op)
    op = GroupedPauli(each_op)
    tmp_cir = Circuit([GroupedPauli(each_op).on(range(8))])
    matrix = tmp_cir.matrix()
    

0 YZIYYXXY
1 IZZYYXZI
2 XIIXYIIZ
3 IIIYYIYX


In [22]:

def generate_ghz_state(num_bits):
    # 创建一个包含 2^num_bits 个元素的零向量
    ghz_state = np.zeros((2**num_bits,), dtype=np.complex128)
    
    # 设置第一个元素为 |000...0>，第二个元素为 |111...1>
    ghz_state[0] = 1  # |000...0>
    ghz_state[-1] = 1  # |111...1>
    
    # 归一化
    ghz_state /= np.sqrt(2)
    
    return ghz_state

ghz_8_qbits = generate_ghz_state(8)

In [123]:
class ForwardAndLoss(ms.nn.Cell):
    def __init__(self, backbone, loss_fn):
        super(ForwardAndLoss, self).__init__(auto_prefix=False)
        self.backbone = backbone
        self.loss_fn = loss_fn

    def construct(self, data, label):
        output = self.backbone(data)
        return self.loss_fn(output, label)

    def backbone_network(self):
        return self.backbone


In [131]:
from mindspore import nn
def Mindspore_ansatz(Structure_p:Parameter,Ansatz_p:Parameter,n_layer:int,n_qbits:int=8):
    """
    和 DQAS 文章描述的一致，生成权重线路
    Structure_p:np.array DQAS中的权重参数,
    Ansatz_p:np.array  DQAS中的Ansatz参数,
    
    """
    Structure_p = Parameter(Tensor(Structure_p, ms.float32), requires_grad=True)
    #Ansatz_p = Parameter(Tensor(Ansatz_p, ms.float32), requires_grad=True)
    softmax = ops.Softmax()
    my_stp = softmax(Tensor(Structure_p, ms.float32))
    ansatz = Circuit()
    pr_gen = PRGenerator('ansatz')
    
    for i in range(n_layer):
        paramertized_part_count=0
        for index_op,each_op in enumerate(unbound_opeartor_pool):
            # ansatz_param = Ansatz_p[i,index_op]
            #Structure_param =float(stp[i,index_op])
            ansatz += TimeEvolution(QubitOperator(terms=each_op,coefficient=pr_gen.new()),time=float(my_stp[i,index_op])).circuit
            paramertized_part_count+=1
            
        for index_op,each_op in enumerate(bound_opeartor_pool):
            #print(index_op,each_op,paramertized_part_count)
            op = GroupedPauli(each_op)
            tmp_cir = Circuit([GroupedPauli(each_op).on(range(n_qbits))])
            matrix = tmp_cir.matrix()
            #print(matrix.shape,my_stp[i,index_op+paramertized_part_count])
            ansatz += UnivMathGate(matrix_value=matrix*float(my_stp[i,index_op+paramertized_part_count]),name=op.pauli_string).on(range(n_qbits))  
            
    sim = Simulator(backend='mqvector',n_qubits=8)
    hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0,1]]
    grad_ops= sim.get_expectation_with_grad(hams,ansatz)
    QuantumNet = MQLayer(grad_ops)   
    loss = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean') 
    net_with_loss = ForwardAndLoss(QuantumNet, loss)
    # grad = ms.ops.GradOperation(get_by_list=True)
    
    
        # 计算 loss 和 ansatz_parameter 的梯度
    class GradientCalculator(nn.Cell):
        def __init__(self, network, weights):
            super(GradientCalculator, self).__init__()
            self.network = network
            self.weights = weights
            self.grad = ops.GradOperation(get_by_list=True)

        def construct(self, data, label):
            loss = self.network(data, label)
            grads = self.grad(self.network, self.weights)(data, label)
            return loss, grads

    # 定义结构参数和 ansatz 参数的权重
    weights = [Parameter(Tensor(Ansatz_p, ms.float32), requires_grad=True)]
    grad_calculator = GradientCalculator(net_with_loss, weights)
    
    # loss_value = net_with_loss(X_train, y_train)
    
    # grads = grad(self.network, weights)(data, label)
    
    return ansatz, grad_calculator
            
    # sim = Simulator(backend='mqvector',n_qubits=8)
    # sim.apply_circuit(ansatz,pr=Ansatz_p.reshape(-1))
    # state =  sim.get_qs()
    # target_tensor = ms.Tensor(ghz_8_qbits, dtype=ms.float32)
    # state_tensor = ms.Tensor(state, dtype=ms.float32)
    # loss = sum(ops.abs(target_tensor-state_tensor))
    # return loss
    


In [135]:
ansatz,gr = Mindspore_ansatz(Structure_p=stp,Ansatz_p=nnp,n_layer=3,n_qbits=8)

In [138]:
# 假设 X_train 和 y_train 已经定义好
loss_value, ansatz_grads = gr(Tensor(nnp, ms.float32), Tensor(y_train, ms.int32))


ValueError: Encoder data requires a two dimension Tensor with second dimension should be 0, but get shape (3, 8)

In [ ]:
g

In [99]:


def fn(x, y, z):
    res = x * ops.exp(y) * ops.pow(z, 2)
    return res, z

x = Tensor([3, 3], ms.float32)
y = Tensor([0, 0], ms.float32)
z = Tensor([5, 5], ms.float32)
gradient, aux = grad(fn, (1, 2), None, True,True)(x, y, z)
gradient

((1,
  Tensor(shape=[2], dtype=Float32, value= [ 7.50000000e+01,  7.50000000e+01])),
 (2,
  Tensor(shape=[2], dtype=Float32, value= [ 3.00000000e+01,  3.00000000e+01])))

In [103]:
aux

(Tensor(shape=[2], dtype=Float32, value= [ 5.00000000e+00,  5.00000000e+00]),)

In [ ]:
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)

def wrapped_mindspore_ansatz(Structure_p, Ansatz_p, n_layer=3, n_qbits=8):
    return Mindspore_ansatz(Structure_p, Ansatz_p, n_layer=n_layer, n_qbits=n_qbits)

# 创建带梯度计算的函数
vag2 = ops.value_and_grad(fn=wrapped_mindspore_ansatz, grad_position=(0, 1))

# 使用 vag2 计算损失及其对 Structure_p 和 Ansatz_p 的梯度
loss_value, (grad_structure, grad_ansatz) = vag2(stp, nnp)  # 确保 stp 和 nnp 是 numpy array
print("Loss Value:", loss_value)
print("Gradient for Structure_p:", grad_structure)
print("Gradient for Ansatz_p:", grad_ansatz)



ValueError: not enough values to unpack (expected 2, got 0)

In [94]:
Mindspore_ansatz(stp,nnp,3,8)

Tensor(shape=[], dtype=Float32, value= 1.41421)

In [67]:
vag2(stp,nnp,3,8)

(Tensor(shape=[], dtype=Float32, value= 1.00708), ())

In [49]:
import mindspore as ms
import mindspore.ops as ops
from mindspore import Tensor, Parameter
from mindquantum import Circuit, Simulator, QubitOperator, TimeEvolution, UnivMathGate

# 定义 GHZ 状态

ghz_8_qbits = generate_ghz_state(8)
num_layer = 3
# 定义标准差和形状
stddev = 0.02
shape_nnp = (num_layer, 8)
shape_stp = (num_layer, 12)

rtype = np.float64
ctype = np.complex128
# 使用 numpy 生成随机数矩阵
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)

def Mindspore_ansatz(Structure_p: Parameter, Ansatz_p: Parameter, n_layer: int=3, n_qbits: int = 8):
    softmax = ops.Softmax()
    my_stp = softmax(Tensor(Structure_p, ms.float32))
    ansatz = Circuit()

    for i in range(n_layer):
        paramertized_part_count = 0
        for index_op, each_op in enumerate(unbound_opeartor_pool):
            ansatz += TimeEvolution(QubitOperator(terms=each_op, coefficient=float(Ansatz_p[i, index_op])), 
                                   time=float(my_stp[i, index_op])).circuit
            paramertized_part_count += 1

        for index_op, each_op in enumerate(bound_opeartor_pool):
            op = GroupedPauli(each_op)
            tmp_cir = Circuit([GroupedPauli(each_op).on(range(n_qbits))])
            matrix = tmp_cir.matrix()
            ansatz += UnivMathGate(matrix_value=matrix * float(my_stp[i, index_op + paramertized_part_count]), 
                                   name=op.pauli_string).on(range(n_qbits))

    # 使用模拟器应用电路
    sim = Simulator(backend='mqvector', n_qubits=n_qbits)
    sim.apply_circuit(ansatz)
    state = sim.get_qs()

    # 计算损失
    target_tensor = Tensor(ghz_8_qbits, dtype=ms.float32)
    state_tensor = Tensor(state, dtype=ms.float32)
    loss = ops.ReduceSum(ops.Abs(target_tensor - state_tensor))

    return loss

# 确保参数被定义为 Parameter 类型，并设置 requires_grad=True
Structure_p = Parameter(Tensor(stp), requires_grad=True)  # 你的结构参数
Ansatz_p = Parameter(Tensor(nnp), requires_grad=True)      # 你的 Ansatz 参数

# 使用 value_and_grad 来计算损失和梯度
vag2 = ms.value_and_grad(Mindspore_ansatz, grad_position=(0, 1))

# 使用示例
loss_value, gradients = vag2(Structure_p, Ansatz_p, n_layer=3, n_qbits=8)

# print("Loss:", loss_value)
# print("Gradients:", gradients)


TypeError: too many positional arguments